ETL:

Extract 

In [1]:
import json

import pandas as pd

import numpy as np 

In [2]:
#load json file as list of dicts to clean (not df bcs not flat, aka messy)
# with statement to handle file resource w mode r for read; read into variable file; 
# use json.load() to save data to new var; wiki movies raw is list of dicts

with open("..\Resources\wikipedia-movies.json", mode = "r") as file: 
    wiki_movies_raw = json.load(file)

In [3]:
#check num of dicts in list 
len(wiki_movies_raw)

7311

In [4]:
#check sample w index slices (use case for negative index slices)

#beginning 
wiki_movies_raw[:5]

#end
wiki_movies_raw[-5:]

#mid
wiki_movies_raw[3000:3005]

[{'url': 'https://en.wikipedia.org/wiki/Human_Nature_(2001_film)',
  'year': 2001,
  'imdb_link': 'https://www.imdb.com/title/tt0219822/',
  'title': 'Human Nature',
  'Directed by': 'Michel Gondry',
  'Produced by': ['Anthony Bregman',
   'Ted Hope',
   'Spike Jonze',
   'Charlie Kaufman'],
  'Written by': 'Charlie Kaufman',
  'Starring': ['Tim Robbins',
   'Patricia Arquette',
   'Rhys Ifans',
   'Miranda Otto',
   'Rosie Perez'],
  'Music by': 'Graeme Revell',
  'Cinematography': 'Tim Maurice-Jones',
  'Edited by': 'Russell Icke',
  'Productioncompany ': ['StudioCanal', 'Good Machine'],
  'Distributed by': ['Fine Line Features',
   '(United States)',
   'BAC Films',
   '(France)'],
  'Release date': ['May 18, 2001',
   '(',
   '2001-05-18',
   ')',
   '(',
   'Cannes',
   ')',
   'September 12, 2001',
   '(',
   '2001-09-12',
   ')',
   '(France)',
   'April 12, 2002',
   '(',
   '2002-04-12',
   ')',
   '(United States)'],
  'Running time': '96 minutes',
  'Country': ['United State

In [5]:
#load kaggle meta and rating data; confirmed flat so load directly into df 

metadata = pd.read_csv("..\Resources\movies_metadata.csv", low_memory=False)

ratings = pd.read_csv("../Resources/ratings.csv")

In [6]:
metadata.sample(n=5)

ratings.sample(n=3)

,userId,movieId,rating,timestamp
15723426,163536,480,4.0,945473669
699120,7127,3686,2.5,1385111678
14739965,153303,3615,2.5,1395706880


Transform

iterative: inspect, plan, execute 

In [7]:
# inspect w df 

wiki_movies_df = pd.DataFrame(wiki_movies_raw)

wiki_movies_df.head()

,url,year,imdb_link,title,Directed by,Produced by,Screenplay by,Story by,Based on,Starring,...,Predecessor,Founders,Area served,Products,Services,Russian,Hebrew,Revenue,Operating income,Polish
0,https://en.wikipedia.org/wiki/The_Adventures_o...,1990.0,https://www.imdb.com/title/tt0098987/,The Adventures of Ford Fairlane,Renny Harlin,"[Steve Perry, Joel Silver]","[David Arnott, James Cappe, Daniel Waters]","[David Arnott, James Cappe]","[Characters, by Rex Weiner]","[Andrew Dice Clay, Wayne Newton, Priscilla Pre...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"https://en.wikipedia.org/wiki/After_Dark,_My_S...",1990.0,https://www.imdb.com/title/tt0098994/,"After Dark, My Sweet",James Foley,"[Ric Kidney, Robert Redlin]","[James Foley, Robert Redlin]",NaN,"[the novel, After Dark, My Sweet, by, Jim Thom...","[Jason Patric, Rachel Ward, Bruce Dern, George...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://en.wikipedia.org/wiki/Air_America_(film),1990.0,https://www.imdb.com/title/tt0099005/,Air America,Roger Spottiswoode,Daniel Melnick,"[John Eskow, Richard Rush]",NaN,"[Air America, by, Christopher Robbins]","[Mel Gibson, Robert Downey Jr., Nancy Travis, ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://en.wikipedia.org/wiki/Alice_(1990_film),1990.0,https://www.imdb.com/title/tt0099012/,Alice,Woody Allen,Robert Greenhut,NaN,NaN,NaN,"[Alec Baldwin, Blythe Danner, Judy Davis, Mia ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://en.wikipedia.org/wiki/Almost_an_Angel,1990.0,https://www.imdb.com/title/tt0099018/,Almost an Angel,John Cornell,John Cornell,NaN,NaN,NaN,"[Paul Hogan, Elias Koteas, Linda Kozlowski]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
wiki_movies_df.columns.to_list()

['url',
 'year',
 'imdb_link',
 'title',
 'Directed by',
 'Produced by',
 'Screenplay by',
 'Story by',
 'Based on',
 'Starring',
 'Narrated by',
 'Music by',
 'Cinematography',
 'Edited by',
 'Productioncompany ',
 'Distributed by',
 'Release date',
 'Running time',
 'Country',
 'Language',
 'Budget',
 'Box office',
 'Written by',
 'Genre',
 'Theme music composer',
 'Country of origin',
 'Original language(s)',
 'Producer(s)',
 'Editor(s)',
 'Production company(s)',
 'Original network',
 'Original release',
 'Productioncompanies ',
 'Executive producer(s)',
 'Production location(s)',
 'Distributor',
 'Picture format',
 'Audio format',
 'Voices of',
 'Followed by',
 'Composer(s)',
 'Created by',
 'Also known as',
 'Opening theme',
 'No. of episodes',
 'Preceded by',
 'Author',
 'Publisher',
 'Publication date',
 'Media type',
 'Pages',
 'ISBN',
 'OCLC',
 'LC Class',
 'Cover artist',
 'Series',
 'Set in',
 'Adaptation by',
 'Suggested by',
 'Biographical data',
 'Born',
 'Died',
 'Resti

In [9]:
#list comp to filter
# dir and imbd filter removed ~100, count now 7080

wiki_movies = [movie for movie in wiki_movies_raw
                if ('Director' in movie or 'Directed by' in movie)
                and 'imdb_link' in movie]

len(wiki_movies)

7080

In [10]:
#re-create df - columns down from 198 to 78
wiki_movies_df = pd.DataFrame(wiki_movies)

wiki_movies_df.columns.to_list()

['url',
 'year',
 'imdb_link',
 'title',
 'Directed by',
 'Produced by',
 'Screenplay by',
 'Story by',
 'Based on',
 'Starring',
 'Narrated by',
 'Music by',
 'Cinematography',
 'Edited by',
 'Productioncompany ',
 'Distributed by',
 'Release date',
 'Running time',
 'Country',
 'Language',
 'Budget',
 'Box office',
 'Written by',
 'Genre',
 'Theme music composer',
 'Country of origin',
 'Original language(s)',
 'Producer(s)',
 'Editor(s)',
 'Production company(s)',
 'Original network',
 'Original release',
 'Productioncompanies ',
 'Executive producer(s)',
 'Production location(s)',
 'Distributor',
 'Picture format',
 'Audio format',
 'Voices of',
 'Followed by',
 'Composer(s)',
 'Created by',
 'Also known as',
 'Opening theme',
 'No. of episodes',
 'Preceded by',
 'Adaptation by',
 'Suggested by',
 'Traditional',
 'Mandarin',
 'Released',
 'Recorded',
 'Venue',
 'Length',
 'Label',
 'Director',
 'Producer',
 'Animation by',
 'Color process',
 'Simplified',
 'French',
 'Camera setup'

In [11]:
#filter for  'No. of episodes'
#creating new list comp here for demo, would often just go back to edit first list comp in workflow

#len 7076 - removed 4 tv shows 

wiki_movies = [movie for movie in wiki_movies_raw
                if ('Director' in movie or 'Directed by' in movie)
                and 'imdb_link' in movie
                and 'No. of episodes' not in movie]

len(wiki_movies)


7076

Functions for cleaning:

8.3.5

In [12]:

def clean_movie(movie):
    movie = dict(movie) #create local non distructive copy (constructor fxn)
    return movie


In [13]:
#inspect languages 
wiki_movies_df[wiki_movies_df['Arabic'].notnull()]

#found that lang columns are for alt names in each lang -> combine into dict in one column 

,url,year,imdb_link,title,Directed by,Produced by,Screenplay by,Story by,Based on,Starring,...,Hepburn,Literally,Cantonese,Chinese,Yiddish,Arabic,Romanized,Russian,Hebrew,Polish
6838,https://en.wikipedia.org/wiki/The_Insult_(film),2018,https://www.imdb.com/title/tt7048622/,The Insult,Ziad Doueiri,"[Rachid Bouchareb, Jean Bréhat, Julie Gayet, A...",NaN,NaN,NaN,"[Adel Karam, Kamel El Basha]",...,NaN,Case No. 23,NaN,NaN,NaN,قضية رقم ٢٣,Qadiyya raqm 23,NaN,NaN,NaN
7062,https://en.wikipedia.org/wiki/Capernaum_(film),2018,https://www.imdb.com/title/tt8267604/,Capernaum,Nadine Labaki,"[Michel Merkt, Khaled Mouzanar]","[Nadine Labaki, Jihad Hojaily, Michelle Keserw...","[Georges Khabbaz, Nadine Labaki, Michelle Kese...",NaN,"[Zain Al Rafeea, Yordanos Shiferaw, Boluwatife...",...,NaN,NaN,NaN,NaN,NaN,کفرناحوم‎,NaN,NaN,NaN,NaN


In [14]:
#check columns for alt titles

sorted(wiki_movies_df.columns.to_list())

['Adaptation by',
 'Also known as',
 'Animation by',
 'Arabic',
 'Audio format',
 'Based on',
 'Box office',
 'Budget',
 'Camera setup',
 'Cantonese',
 'Chinese',
 'Cinematography',
 'Color process',
 'Composer(s)',
 'Country',
 'Country of origin',
 'Created by',
 'Directed by',
 'Director',
 'Distributed by',
 'Distributor',
 'Edited by',
 'Editor(s)',
 'Executive producer(s)',
 'Followed by',
 'French',
 'Genre',
 'Hangul',
 'Hebrew',
 'Hepburn',
 'Japanese',
 'Label',
 'Language',
 'Length',
 'Literally',
 'Mandarin',
 'McCune–Reischauer',
 'Music by',
 'Narrated by',
 'No. of episodes',
 'Opening theme',
 'Original language(s)',
 'Original network',
 'Original release',
 'Original title',
 'Picture format',
 'Polish',
 'Preceded by',
 'Produced by',
 'Producer',
 'Producer(s)',
 'Production company(s)',
 'Production location(s)',
 'Productioncompanies ',
 'Productioncompany ',
 'Recorded',
 'Release date',
 'Released',
 'Revised Romanization',
 'Romanized',
 'Running time',
 'Russ

Identify alternate title lang 

Also known as 
Arabic
Cantonese
Chinese
French
Hangul
Hebrew
Japanese
Literally
Mandarin
McCune–Reischauer
Original title
Polish
Revised Romanization
Romanized
Russian
Simplified
Traditional
Yiddish

In [19]:
#handle alt titles 
#pseud:
    # 1 empty dict to hold all alt titles
    # 2 loop thru list of alt tittle keys (check if key exists, add to alt titles dict)
    #3 add alt titles dict to movie object


def clean_movie(movie):
    movie = dict(movie) #create local non distructive copy (constructor fxn)
    alt_titles = {}
    for key in ["Also known as", "Arabic","Cantonese","Chinese","French", "Hepburn",        #1
    "Hangul","Hebrew","Japanese","Literally","Mandarin","McCune–Reischauer",
    "Original title","Polish","Revised Romanization","Romanized","Russian",
    "Simplified","Traditional","Yiddish"]:
        if key in movie:
            alt_titles[key]= movie[key]                                             #2
            movie.pop(key)

    if len(alt_titles) > 0:                                                         #3
        movie['alt_titles'] = alt_titles
        
    return movie


clean_movies = [clean_movie(movie) for movie in wiki_movies]

wiki_movies_df = pd.DataFrame(clean_movies)

sorted(wiki_movies_df.columns.to_list())

['Adaptation by',
 'Animation by',
 'Audio format',
 'Based on',
 'Box office',
 'Budget',
 'Cinematography',
 'Color process',
 'Composer(s)',
 'Country',
 'Country of origin',
 'Created by',
 'Directed by',
 'Director',
 'Distributed by',
 'Distributor',
 'Edited by',
 'Editor(s)',
 'Executive producer(s)',
 'Followed by',
 'Genre',
 'Label',
 'Language',
 'Length',
 'Music by',
 'Narrated by',
 'Original language(s)',
 'Original network',
 'Original release',
 'Picture format',
 'Preceded by',
 'Produced by',
 'Producer',
 'Producer(s)',
 'Production company(s)',
 'Production location(s)',
 'Productioncompanies ',
 'Productioncompany ',
 'Recorded',
 'Release date',
 'Released',
 'Running time',
 'Screen story by',
 'Screenplay by',
 'Starring',
 'Story by',
 'Suggested by',
 'Theme music composer',
 'Venue',
 'Voices of',
 'Written by',
 'alt_titles',
 'imdb_link',
 'title',
 'url',
 'year']